## Setup

In [1]:
%load_ext autoreload
%autoreload 2

### Dependencies

In [2]:
import datetime as dt
import os

import ipywidgets as widgets
from IPython.display import display,clear_output

In [3]:
import pandas as pd
pd.options.mode.chained_assignment = None   # suppresses SettingWithCopyWarning

In [4]:
pwd

'C:\\Users\\timot\\Desktop\\Mila\\Courses\\IFT6758\\Project\\Project Service\\Docker\\notebooks'

In [5]:
from ift6758.client import GameTracker, NHLAPIClient, PredictorAPIClient
from ift6758.client.nhl_api_client import UnknownGameException
from ift6758.client.predictor_api_client import UnknownModelException
from ift6758.client.utils import MODEL_OPTIONS

### Setup

In [6]:
if not os.path.exists("./tmp_data/"):
    os.makedirs("./tmp_data/")

In [7]:
nhl_api = NHLAPIClient(raw_game_directory="./tmp_data/")
predictor = PredictorAPIClient(ip="serving")
#predictor = PredictorAPIClient(ip="127.0.0.1")
game_tracker = GameTracker(nhl_api_client=nhl_api, predictor_client=predictor)

### Dashboard Actions

In [8]:
def on_button_update_clicked(btn):
    game_error = 0 
    model_error = 0
    with output:
        try:
            game_tracker.update(game_id_tbox.value, model_id_tbox.value)
        except UnknownGameException as uge:
            game_error = 1
        except UnknownModelException as ume:
            model_error = 1

        events = game_tracker.get_events_and_predictions()

        most_recent_event = events.head(1).squeeze()
        home_team = most_recent_event['home_team']
        away_team = most_recent_event['away_team']

        home_xG = round(events.loc[events['shooter_team_name']==home_team, 'predictions'].sum(), 2)
        away_xG = round(events.loc[events['shooter_team_name']==away_team, 'predictions'].sum(), 2)

        clear_output()
        if game_error:
            print(f"Game ID {game_id_tbox.value} not recognized, so dashboard unchanged")
            game_error = 0
        if model_error:
            print(f"Dashboard unchanged because model_id " + str(ume))
            model_error = 0
        
        print(f"Model Loaded: {model_id_tbox.value}")
        print(f"Period {most_recent_event['period']}, {most_recent_event['time_remaining']} Remaining")
        print(f"Home: {home_team} Away: {away_team}")
        print(f"Home G: {most_recent_event['goals_home']} Away G: {most_recent_event['goals_away']}")
        print(f"Home xG: {home_xG} Away xG: {away_xG}")
        print(f"Recent events:")
        display(events[['type','description', 'predictions']].head(15))


In [9]:
game_id_tbox = widgets.Text(description="game_id", width=300, value = "2021020008")
model_id_tbox = widgets.Dropdown(
    options=MODEL_OPTIONS,
    value="xgboost-feats-non-corr",
    description='Number:',
)

button_update = widgets.Button(description="Update Dashboard")

button_update.on_click(on_button_update_clicked)
output = widgets.Output()

display(widgets.HBox((game_id_tbox, model_id_tbox, button_update)))
display(output)

Output()